In [1]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
data = pd.read_csv("final_opponent_and_team_data.csv")
data.head()

,Date,Team,Opponent,Location,WAB,ADJO,ADJD,EFF,EFG%,TO%,OR%,FTR,2P,3P,Opp EFF,Opp EFG%,Opp TO%,Opp OR%,Opp FTR,Opp 2P,Opp 3P,Opponent_score,Team_score,opp_adj_o,opp_adj_d
0,2024-11-04,Duke,Maine,H,0.1,125.3,95.2,130.6,64.3,17.7,35.5,31.7,24-34,11-29,84.3,39.5,19.0,19.4,31.6,18-43,3-14,62,96,108.9,110.8
1,2024-11-08,Duke,Army,H,0.1,124.7,92.3,141.0,61.3,11.3,43.6,25.4,18-33,17-38,81.8,39.1,18.3,23.3,14.1,13-35,8-29,58,100,107.2,119.0
2,2024-11-12,Duke,Kentucky,N,-0.2,106.8,86.4,95.7,42.3,9.3,25.0,23.9,24-47,4-24,102.3,47.6,14.6,23.3,38.1,15-38,10-25,77,72,119.6,88.1
3,2024-11-16,Duke,Wofford,H,-0.1,124.7,56.9,133.7,61.3,15.5,45.2,17.7,14-24,16-38,54.4,28.9,29.5,29.3,5.3,9-24,5-33,35,86,69.8,115.5
4,2024-11-22,Duke,Arizona,A,0.6,111.7,75.9,101.9,50.0,20.7,35.1,21.3,17-36,9-25,81.2,45.3,22.2,16.7,20.8,15-30,6-23,55,69,98.9,90.4


In [3]:
data.dtypes

Date               object
Team               object
Opponent           object
Location           object
WAB               float64
ADJO              float64
ADJD              float64
EFF               float64
EFG%              float64
TO%               float64
OR%               float64
FTR               float64
2P                 object
3P                 object
Opp EFF           float64
Opp EFG%          float64
Opp TO%           float64
Opp OR%           float64
Opp FTR           float64
Opp 2P             object
Opp 3P             object
Opponent_score      int64
Team_score          int64
opp_adj_o         float64
opp_adj_d         float64
dtype: object

In [4]:
data.isnull().sum()

Date              0
Team              0
Opponent          0
Location          0
WAB               0
ADJO              0
ADJD              0
EFF               0
EFG%              0
TO%               0
OR%               0
FTR               0
2P                0
3P                0
Opp EFF           0
Opp EFG%          0
Opp TO%           0
Opp OR%           0
Opp FTR           0
Opp 2P            0
Opp 3P            0
Opponent_score    0
Team_score        0
opp_adj_o         0
opp_adj_d         0
dtype: int64

In [5]:
data[data.isnull().any(axis=1)]

,Date,Team,Opponent,Location,WAB,ADJO,ADJD,EFF,EFG%,TO%,OR%,FTR,2P,3P,Opp EFF,Opp EFG%,Opp TO%,Opp OR%,Opp FTR,Opp 2P,Opp 3P,Opponent_score,Team_score,opp_adj_o,opp_adj_d


In [6]:
data = data.dropna()

In [7]:
def preprocess(data):
    data_clean = data.dropna()
    columns_to_convert = [
        "ADJO",
        "ADJD",
        "EFG%",
        "TO%",
        "OR%",
        "FTR",
        "Opp EFG%",
        "Opp TO%",
        "Opp OR%",
        "Opp FTR",
        "opp_adj_o",
        "opp_adj_d",
    ]
    for col in columns_to_convert:
        data[col] = pd.to_numeric(data[col], errors="coerce")

    ratio_columns = ["2P", "3P", "Opp 2P", "Opp 3P"]

    # Convert existing ratio columns to numerical percentages
    for column in ratio_columns:
        if column in data.columns:  # Check if the column exists in the dataset
            data[column] = (
                data[column]
                .str.split("-")
                .apply(
                    lambda x: (
                        int(x[0]) / int(x[1]) if len(x) == 2 and int(x[1]) != 0 else 0
                    )
                )
            )
    categorical_columns = ["Location"]
    data_encoded = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

    data_encoded = data_encoded.apply(
        lambda x: x.astype(int) if x.dtype == "bool" else x
    )

    return data_encoded

In [8]:
preprocessed_data = preprocess(data)

In [9]:
X = preprocessed_data.drop(
    columns=[
        "Team_score",
        "Opponent_score",
        "Team",
        "Opponent",
        "2P",
        "3P",
        "Opp 2P",
        "Opp 3P",
        "WAB",
        "EFF",
        "Opp EFF",
        "Date",
    ]
)
y = preprocessed_data[["Team_score", "Opponent_score"]]

X = X.fillna(0)

In [10]:
X.dtypes

ADJO          float64
ADJD          float64
EFG%          float64
TO%           float64
OR%           float64
FTR           float64
Opp EFG%      float64
Opp TO%       float64
Opp OR%       float64
Opp FTR       float64
opp_adj_o     float64
opp_adj_d     float64
Location_H      int64
Location_N      int64
dtype: object

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [12]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

In [13]:
model = MLPRegressor(
    hidden_layer_sizes=(32, 16), activation="relu", max_iter=500, random_state=42
)

In [14]:
model.fit(X_train, y_train)

/Users/nrutachoudhari/opt/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(32, 16), max_iter=500, random_state=42)

In [15]:
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

In [16]:
# Ensure that y_test is 2D
y_test = np.array(y_test)

# Check if y_test and y_pred have matching shapes
print(f"Shape of y_test: {y_test.shape}")
print(f"Shape of y_pred: {y_pred.shape}")

# If y_test is 2D (it should be with shape (61, 2)), proceed with the metrics
if y_test.ndim == 2 and y_pred.ndim == 2:
    mae_team = mean_absolute_error(y_test[:, 0], y_pred[:, 0])
    mae_opponent = mean_absolute_error(y_test[:, 1], y_pred[:, 1])

    mse_team = mean_squared_error(y_test[:, 0], y_pred[:, 0])
    mse_opponent = mean_squared_error(y_test[:, 1], y_pred[:, 1])

    r2_team = r2_score(y_test[:, 0], y_pred[:, 0])
    r2_opponent = r2_score(y_test[:, 1], y_pred[:, 1])

    print("Team Score Prediction Metrics:")
    print(f"MAE: {mae_team}")
    print(f"MSE: {mse_team}")
    print(f"R² Score: {r2_team}")

    print("\nOpponent Score Prediction Metrics:")
    print(f"MAE: {mae_opponent}")
    print(f"MSE: {mse_opponent}")
    print(f"R² Score: {r2_opponent}")
else:
    print("The shapes of y_test and y_pred do not match.")

Shape of y_test: (61, 2)
Shape of y_pred: (61, 2)
Team Score Prediction Metrics:
MAE: 78.97570822060536
MSE: 6377.666389291225
R² Score: -6833.628915042071

Opponent Score Prediction Metrics:
MAE: 67.47908325493877
MSE: 4677.363016395422
R² Score: -6199.785725916307


### Using Cross Validation

In [17]:
cv_model = MLPRegressor(
    hidden_layer_sizes=(64, 32), activation="relu", max_iter=500, random_state=42
)

In [18]:
# Team score
y_train_team_cv = y_train[:, 0]
# Opponent Score
y_train_opponent_cv = y_train[:, 1]

In [19]:
# Perform 5-fold cross-validation for Team_score
cv_scores_team = cross_val_score(
    model, X_train, y_train_team_cv, cv=5, scoring="neg_mean_absolute_error"
)

# Perform 5-fold cross-validation for Opponent_score
cv_scores_opponent = cross_val_score(
    model, X_train, y_train_opponent_cv, cv=5, scoring="neg_mean_absolute_error"
)

# Convert negative MAE scores to positive
cv_scores_team = -cv_scores_team
cv_scores_opponent = -cv_scores_opponent

# Print the cross-validation results
print(
    f"Cross-validated MAE for Team Score: {cv_scores_team.mean()} ± {cv_scores_team.std()}"
)
print(
    f"Cross-validated MAE for Opponent Score: {cv_scores_opponent.mean()} ± {cv_scores_opponent.std()}"
)

Cross-validated MAE for Team Score: 0.5077428198835452 ± 0.03608442243200175
Cross-validated MAE for Opponent Score: 0.40836878500876655 ± 0.03964139344652046


In [20]:
# Fit the model on the full training data
cv_model.fit(X_train, y_train)

# Make predictions on the test data
cv_ypred_scaled = cv_model.predict(X_test)

# Inverse transform the predictions for both Team_score and Opponent_score
cv_y_pred = scaler_y.inverse_transform(cv_ypred_scaled)

# Evaluate the model's performance
cv_mae_team = mean_absolute_error(y_test[:, 0], cv_y_pred[:, 0])
cv_mae_opponent = mean_absolute_error(y_test[:, 1], cv_y_pred[:, 1])

cv_mse_team = mean_squared_error(y_test[:, 0], cv_y_pred[:, 0])
cv_mse_opponent = mean_squared_error(y_test[:, 1], cv_y_pred[:, 1])

cv_r2_team = r2_score(y_test[:, 0], cv_y_pred[:, 0])
cv_r2_opponent = r2_score(y_test[:, 1], cv_y_pred[:, 1])

# Print the evaluation metrics
print("\nTeam Score Prediction Metrics:")
print(f"MAE: {cv_mae_team}")
print(f"MSE: {cv_mse_team}")
print(f"R² Score: {cv_r2_team}")

print("\nOpponent Score Prediction Metrics:")
print(f"MAE: {cv_mae_opponent}")
print(f"MSE: {cv_mse_opponent}")
print(f"R² Score: {cv_r2_opponent}")


Team Score Prediction Metrics:
MAE: 79.05576141895446
MSE: 6385.478478391738
R² Score: -6842.000743669374

Opponent Score Prediction Metrics:
MAE: 67.21358634601742
MSE: 4645.350957545257
R² Score: -6157.347301342567


### Hyperparameter tuning on CV

In [21]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    "hidden_layer_sizes": [(32,), (64, 32), (128, 64, 32)],  # Different layer sizes
    "activation": ["relu", "tanh"],  # Activation functions
    "solver": ["adam", "sgd"],  # Solvers
    "alpha": [0.0001, 0.001, 0.01],  # L2 regularization
    "learning_rate_init": [0.001, 0.01],  # Learning rate
    "max_iter": [500],  # Maximum number of iterations
}

# Initialize the MLPRegressor
mlp = MLPRegressor(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=mlp,
    param_grid=param_grid,
    scoring="neg_mean_absolute_error",  # Use MAE as the evaluation metric
    cv=5,  # 5-fold cross-validation
    verbose=2,  # Display progress
    n_jobs=-1,  # Use all available CPU cores
)

# Perform grid search
grid_search.fit(X_train, y_train)

# Display the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best CV Score (MAE):", -grid_search.best_score_)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.1s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.1s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.1s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.1s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.1s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.1s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solve

/Users/nrutachoudhari/opt/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.01, max_iter=500, solver=adam; total time=   0.1s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.01, max_iter=500, solver=adam; total time=   0.1s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   0.1s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   0.1s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   0.1s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   0.1s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(32,), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   0.1s
[CV] END activation=relu, alpha=0.0001, hidden

/Users/nrutachoudhari/opt/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   0.6s
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.1s
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.1s
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.1s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   0.5s
[CV] END activation=relu, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.01, max_iter=500, solver=sgd; total time=   0.6s
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(32,), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.1s
[CV] END activation=rel

...

[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.3s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.3s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.5s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.2s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.7s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.7s
[CV] END activation=tanh, alpha=0.0001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=sgd; t

/Users/nrutachoudhari/opt/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.3s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.2s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.3s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.6s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.6s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.7s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=sgd; total ti

/Users/nrutachoudhari/opt/miniconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=tanh, alpha=0.01, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.6s
[CV] END activation=tanh, alpha=0.01, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.3s
[CV] END activation=tanh, alpha=0.01, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.3s
[CV] END activation=tanh, alpha=0.01, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0.3s
[CV] END activation=tanh, alpha=0.01, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.7s
[CV] END activation=tanh, alpha=0.01, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=adam; total time=   0.8s
[CV] END activation=tanh, alpha=0.01, hidden_layer_sizes=(128, 64, 32), learning_rate_init=0.001, max_iter=500, solver=sgd; total time=   0

In [22]:
# Train the best model
best_model = grid_search.best_estimator_  # or random_search.best_estimator_
best_model.fit(X_train, y_train)

# Predict on the test set
ypred_scaled = best_model.predict(X_test)
y_pred = scaler_y.inverse_transform(ypred_scaled)

# Evaluate performance
mae_team = mean_absolute_error(y_test[:, 0], y_pred[:, 0])
mae_opponent = mean_absolute_error(y_test[:, 1], y_pred[:, 1])

print(f"Test MAE for Team Score: {mae_team}")
print(f"Test MAE for Opponent Score: {mae_opponent}")

Test MAE for Team Score: 79.77829030423078
Test MAE for Opponent Score: 68.02551384956325


### Predict data

In [23]:
predict_data = pd.read_csv("basketball_games_data.csv")
predict_data

,date,Location,Team,Opponent,ADJO,ADJD,EFG%,TO%,OR%,FTR,Opp EFG%,Opp TO%,Opp OR%,Opp FTR,opp_adj_o,opp_adj_d
0,2024-12-23,Home,North Dakota St.,Cal St. Bakersfield,133.4,89.4,62.7,8.8,20.0,68.6,46.3,16.1,21.2,38.9,85.2,125.3
1,2024-12-23,Neutral,Charleston,Loyola Chicago,102.1,91.8,56.0,20.9,9.7,27.6,46.6,24.8,31.6,34.5,95.6,97.9
2,2024-12-23,Neutral,Charlotte,Murray St.,118.6,110.4,50.0,17.4,36.4,60.0,46.4,8.7,28.2,52.2,106.0,115.3
3,2024-12-23,Home,Tennessee,Middle Tennessee,112.6,95.4,57.1,15.7,25.8,41.1,48.3,15.7,15.8,23.7,116.3,98.5
4,2024-12-23,Away,Nebraska,Hawaii,107.5,93.1,53.6,19.3,33.3,20.0,47.9,25.7,32.1,31.9,103.3,103.5
5,2024-12-23,Neutral,Oregon St.,Oakland,110.6,112.1,50.8,16.1,46.2,28.8,50.9,20.5,38.2,42.9,120.8,109.1
6,2024-12-23,Away,Seattle,Washington,120.1,92.5,44.6,11.1,21.9,57.1,47.4,16.7,21.6,40.4,91.8,112.2


In [24]:
predict_data["Location"] = predict_data["Location"].replace(
    {"Home": "H", "Away": "A", "Neutral": "N"}
)

In [25]:
predict_data_process = preprocess(predict_data)
predict_data_process = predict_data_process.drop(columns=["date", "Team", "Opponent"])
predict_data_process = predict_data_process.fillna(0)
predict_data_scaled = scaler_X.transform(predict_data_process)

In [26]:
# predicting using normal NN

nn_predictions_scaled = model.predict(predict_data_scaled)
nn_predictions = scaler_y.inverse_transform(nn_predictions_scaled)

# Add predictions to the original dataset
predict_data["NN_Team_Score"] = nn_predictions[:, 0]
predict_data["NN_Opp_Score"] = nn_predictions[:, 1]

In [27]:
# predicting using MLP with Cross Validation

cv_predictions_scaled = cv_model.predict(predict_data_scaled)
cv_predictions = scaler_y.inverse_transform(cv_predictions_scaled)

# Add predictions to the original dataset
predict_data["CV_Team_Score"] = cv_predictions[:, 0]
predict_data["CV_Opp_Score"] = cv_predictions[:, 1]

In [28]:
# predicting using CV with Hyperparameter Tuning

hpt_predictions_scaled = best_model.predict(predict_data_scaled)
hpt_predictions = scaler_y.inverse_transform(hpt_predictions_scaled)

# Add predictions to the original dataset
predict_data["HPT_Team_Score"] = hpt_predictions[:, 0]
predict_data["HPT_Opp_Score"] = hpt_predictions[:, 1]

In [29]:
# # adding the manual scores manually coz idk
# predict_data["Actual_Team"] = [89, 80, 80, 73, 81, 90, 83, 89]
# predict_data["Actual_Opp"] = [66, 77, 77, 79, 54, 46, 90, 58]

In [29]:
predict_data[
    [
        "Team",
        "Opponent",
        "NN_Team_Score",
        "NN_Opp_Score",
        "CV_Team_Score",
        "CV_Opp_Score",
        "HPT_Team_Score",
        "HPT_Opp_Score",
        # "Actual_Team",
        # "Actual_Opp",
    ]
]

,Team,Opponent,NN_Team_Score,NN_Opp_Score,CV_Team_Score,CV_Opp_Score,HPT_Team_Score,HPT_Opp_Score
0,North Dakota St.,Cal St. Bakersfield,103.048543,62.313430,104.585169,57.022475,94.321366,62.175115
1,Charleston,Loyola Chicago,67.990130,64.501119,63.955724,59.755754,63.368783,60.149350
2,Charlotte,Murray St.,81.753652,81.271724,80.319871,81.560754,82.269805,78.361510
3,Tennessee,Middle Tennessee,79.710481,70.550996,81.096270,71.565305,85.452788,72.658354
4,Nebraska,Hawaii,72.128147,64.660960,70.993944,59.203223,70.871939,61.292500
5,Oregon St.,Oakland,80.348559,78.478586,89.228236,81.505253,80.415674,78.985394
6,Seattle,Washington,78.110364,66.578955,73.424942,67.106199,76.634823,63.377189


In [31]:
predict_data[
    [
        "Team",
        "Opponent",
        "NN_Team_Score",
        "NN_Opp_Score",
        "CV_Team_Score",
        "CV_Opp_Score",
        "HPT_Team_Score",
        "HPT_Opp_Score",
        "Actual_Team",
        "Actual_Opp",
    ]
].to_csv("mlp_predictions.csv")